# மாதிரி 01: OpenAI SDK மூலம் விரைவான உரையாடல்

இந்த நோட்புக் OpenAI SDK-ஐ Microsoft Foundry Local உடன் இணைத்து விரைவான உரையாடல்களை எப்படி செய்யலாம் என்பதை விளக்குகிறது.

## மேலோட்டம்

இந்த உதாரணம் காட்டுகிறது:
- Foundry Local உடன் OpenAI Python SDK-ஐ பயன்படுத்துவது
- Azure OpenAI மற்றும் உள்ளூர் Foundry அமைப்புகளை கையாளுவது
- சரியான பிழை கையாளுதல் மற்றும் மாற்று உத்திகளை செயல்படுத்துவது
- FoundryLocalManager-ஐ சேவை மேலாண்மைக்காக பயன்படுத்துவது


## முன்பதிவுகள்

உங்களிடம் இருக்க வேண்டும்:
1. Foundry Local நிறுவப்பட்டு இயக்கப்பட வேண்டும்
2. ஒரு மாடல் ஏற்றப்பட்டிருக்க வேண்டும் (எ.கா., `foundry model run phi-4-mini`)
3. தேவையான Python தொகுப்புகள் நிறுவப்பட்டிருக்க வேண்டும்

### சார்புகளை நிறுவவும்


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## தேவையான நூலகங்களை இறக்குமதி செய்யவும்


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## கட்டமைப்பு

உங்கள் கட்டமைப்பை அமைக்கவும். நீங்கள் பின்வருவனவற்றில் ஒன்றை பயன்படுத்தலாம்:
1. **Azure OpenAI** (மேக அடிப்படையிலானது)
2. **Foundry Local** (உள்ளூர் முடிவீடு)

### விருப்பம் 1: Azure OpenAI கட்டமைப்பு


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### விருப்பம் 2: ஃபவுண்ட்ரி உள்ளூர் கட்டமைப்பு


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## கிளையன்ட் ஆரம்பித்தல்

உங்கள் கட்டமைப்பின் அடிப்படையில் OpenAI கிளையன்டை உருவாக்கவும்:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## அடிப்படை உரையாடல் உதாரணம்

ஒரு எளிய உரையாடல் தொடர்பை சோதிப்போம்:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## தொடர்பு கொண்ட உரையாடல் அமர்வு

மாதிரியின் பதில்களைப் பார்க்க பல்வேறு வகையான உந்துதல்களை முயற்சிக்கவும்:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## மேம்பட்ட பயன்பாடு: ஸ்ட்ரீமிங் பதில்கள்

நீண்ட பதில்களுக்கு, ஸ்ட்ரீமிங் ஒரு சிறந்த பயனர் அனுபவத்தை வழங்க முடியும்:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## பிழை கையாளல் மற்றும் கண்டறிதல்

சேவை நிலை மற்றும் கிடைக்கக்கூடிய மாதிரிகளை சரிபார்ப்போம்:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## தனிப்பயன் உத்தரவுகள் சோதனை

கீழே உள்ள செல் மூலம் உங்கள் சொந்த உத்தரவுகளை சோதிக்கவும்:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## சுருக்கம்

இந்த நோட்புக் காட்டியது:

1. **✅ கிளையன்ட் அமைப்பு**: Foundry Local உடன் OpenAI SDK ஐ எப்படி அமைப்பது
2. **✅ அடிப்படை உரையாடல்**: எளிய கோரிக்கை-பதில் தொடர்புகள்
3. **✅ ஸ்ட்ரீமிங்**: நேரடி பதில் உருவாக்கம்
4. **✅ பிழை கையாளுதல்**: வலுவான பிழை கையாளுதல் மற்றும் கண்டறிதல்
5. **✅ சேவை நிலை**: மாடல் கிடைக்கும் நிலை மற்றும் சேவை நிலையை சரிபார்த்தல்

### அடுத்த படிகள்

- **Sample 02**: Azure OpenAI ஆதரவு கொண்ட மேம்பட்ட SDK ஒருங்கிணைப்பு
- **Sample 04**: Chainlit உரையாடல் பயன்பாட்டை உருவாக்குதல்
- **Sample 05**: பல முகவர்களுடன் ஒருங்கிணைப்பு அமைப்புகள்
- **Sample 06**: புத்திசாலி மாடல் வழிமுறைகள்

### Foundry Local இன் முக்கிய நன்மைகள்

- 🔒 **தனியுரிமை**: தரவுகள் உங்கள் சாதனத்தை விட்டு வெளியே செல்லாது
- ⚡ **வேகம்**: குறைந்த தாமத உள்ளூர் முடிவு
- 💰 **செலவு**: API பயன்பாட்டு செலவுகள் இல்லை
- 🔌 **ஆஃப்லைன்**: இணைய இணைப்பு இல்லாமல் செயல்படும்
- 🛠️ **இணக்கத்தன்மை**: OpenAI-இன் இணக்கமான API



---

**அறிவிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையை பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளவும். அதன் சொந்த மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்களுக்கும் அல்லது தவறான விளக்கங்களுக்கும் நாங்கள் பொறுப்பல்ல.
